In [2]:
import sys
sys.path.append("..")  

from openbb_terminal.sdk import openbb
from backend.financialmodelingprep import fmp_stocks
import yaml
from datetime import datetime
import pandas as pd
import os
from backend.db import LogAvailableTickers, LogIngestedTickers,Balance,  bulk_insert_data_from_dataframe, read_table_data, engine

from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

%load_ext autoreload
%autoreload 2


INFO:openbb_terminal.cryptocurrency.onchain.bitquery_model:START
INFO:openbb_terminal.cryptocurrency.onchain.bitquery_model:END


Forecasting Toolkit is disabled. To use the Forecasting features please install the toolkit following the instructions here: https://my.openbb.co/app/sdk/installation

Portfolio Optimization Toolkit is disabled. To use the Optimization features please install the toolkit following the instructions here: https://my.openbb.co/app/sdk/installation

INFO:openbb_terminal.loggers:Logging configuration finished
INFO:openbb_terminal.loggers:Logging set to ['file', 'posthog']
INFO:openbb_terminal.loggers:Verbosity set to 20
INFO:openbb_terminal.loggers:LOGFORMAT: %(levelname)s-%(appName)s-%(commitHash)s-%(appId)s-%(sessionId)s-%(userId)s-%(asctime)s-%(name)s-%(funcName)s-%(lineno)s-%(message)s
INFO:openbb_terminal.core.log.generation.settings_logger:STARTUP: {"PREFERENCES": {"PLOT_BACKEND": null, "PLOT_DPI": 100, "PLOT_HEIGHT": 500, "PLOT_WIDTH": 800, "PLOT_HEIGHT_PERCENTAGE": 50.0, "PLOT_WIDTH_PERCENTAGE": 70.0, "PLOT_OPEN_EXPORT": false, "PLOT_ENABLE_PYWRY": true, "PLOT_PYWRY_WIDTH": 1400, "PLOT_PYWRY_HEIGHT": 762, "FILE_OVERWRITE": false, "SHOW_VERSION": true, "RETRY_WITH_LOAD": false, "USE_TABULATE_DF": true, "USE_INTERACTIVE_DF": true, "USE_CLEAR_AFTER_CMD": false, "USE_DATETIME": true, "USE_PROMPT_TOOLKIT": true, "USE_PLOT_AUTOSCALING": false, "ENABLE_THOUGHTS_DAY": false, "ENABLE_QUICK_EXIT": false, "OPEN_REPORT_AS_HTML": true, 

In [3]:

database_path = os.getenv("DATABASE_PATH")
fmp_api_aky = os.getenv("FMP_SECRET_KEY")

In [4]:
# Fetch the companies and tickers
openbb.keys.fmp(key=fmp_api_aky, persist=True)
companies = fmp_stocks.available_companies(fmp_api_aky)
companies_nasdaq = companies[(companies['exchangeShortName'] == "NASDAQ") & (companies['type'] == "stock")]
companies_milan= companies[(companies['exchangeShortName'] == "MIL") & (companies['type'] == "stock")]

list_ticker_nasdaq = companies_nasdaq.symbol.to_list()
list_ticker_mil = companies_milan.symbol.to_list()

INFO:openbb_terminal.keys_model:START
INFO:openbb_terminal.keys_model:{"INPUT": {"persist": "True", "show_output": "False", "chart": "False"}, "VIRTUAL_PATH": "keys.fmp", "CHART": false}
INFO:openbb_terminal.keys_model:END


In [5]:
list_ticker = list_ticker_nasdaq

In [6]:

df_log = pd.DataFrame([{
    'timestamp': datetime.now(),
    'num_companies': len(companies),
    'num_tickers': len(list_ticker)
}]) 

In [7]:
bulk_insert_data_from_dataframe(LogAvailableTickers, df_log)

In [8]:

# Partiamo da una lista di ticker ridotta e iteriamo su 100 trimestri. Sicuramente c'è un modo piu ottimale di farlo, ma al momento lo lasciamo cosi
list_ticker = ['AAPL', "PLTR","AMZN","META","NIO","RGTI","FFIEW","NVDA","NFLX","BABA"]

source="FinancialModelingPrep"
quarterly=True
limit=100

# Leggiamo i dati già storicizzati
df = read_table_data("fa_balance")   

In [9]:
df

,fiscal_date_ending,reported_currency,cik,filling_date,accepted_date,calendar_year,period,cash_and_cash_equivalents,short_term_investments,cash_and_short_term_investments,...,total_liabilities_and_stockholders_equity,minority_interest,total_liabilities_and_total_equity,total_investments,total_debt,net_debt,link,final_link,ticker,current_date


In [10]:
def process_ticker(ticker, df):
    df_quarterly = openbb.stocks.fa.balance(symbol=ticker, source=source, quarterly=quarterly, limit=limit)
    
    if df_quarterly.empty:
        print(f"Ticker {ticker} not found")
        return df, None
    
    df_quarterly = df_quarterly.T.reset_index()
    df_quarterly['ticker'] = ticker
    df_quarterly['current_date'] = pd.Timestamp.now().floor('S')
    df_quarterly.columns = [col.lower().replace(" ","_") for col in df_quarterly.columns]
    
    # Convert both 'calendar_year' columns to datetime
    df_quarterly['calendar_year'] = pd.to_datetime(df_quarterly['calendar_year'])
    if 'calendar_year' in df.columns:
        df['calendar_year'] = pd.to_datetime(df['calendar_year'])
    
    if df.empty:
        
        df_log_process = pd.DataFrame([{

            'ingestion_timestamp': datetime.now(),
            'ticker': ticker,
            'num_new_rows': len(df_quarterly),
            'num_old_rows_diff_dates': 0,
            'num_columns': len(df_quarterly.columns)
        
        }])
            
            
        
        return df_quarterly, df_log_process
    
    df_quarterly_merge = df_quarterly.merge(df, how='left', indicator=True, on=['cik', 'calendar_year', 'period'], suffixes=('', '_y'))
    df_new_rows = df_quarterly_merge[df_quarterly_merge['_merge'] == 'left_only']
    df_old_rows = df_quarterly_merge[df_quarterly_merge['_merge'] == 'both']
    df_old_rows_diff_dates = df_old_rows[(df_old_rows['filling_date'] != df_old_rows['filling_date_y']) | (df_old_rows['accepted_date'] != df_old_rows['accepted_date_y'])]
    
    df_log_process = pd.DataFrame([{
        'ingestion_timestamp': datetime.now(),
        'ticker': ticker,
        'num_new_rows': len(df_new_rows),
        'num_old_rows_diff_dates': len(df_old_rows_diff_dates),
        'num_columns': len(df_quarterly.columns)
    }])
    

    return pd.concat([df_new_rows[df.columns], df_old_rows_diff_dates[df.columns]]), df_log_process

In [11]:
# Main script
dataframes = []
records = []

for ticker in list_ticker:
    df_processed, df_log = process_ticker(ticker, df)
    if df_processed is not None:
        dataframes.append(df_processed)
    if df_log is not None:
        bulk_insert_data_from_dataframe(LogIngestedTickers, df_log)

df_final = pd.concat(dataframes, ignore_index=True)

date_columns = ['filling_date', 'accepted_date', 'current_date']
for col in date_columns:
    df_final[col] = pd.to_datetime(df_final[col])

INFO:openbb_terminal.stocks.fundamental_analysis.sdk_helpers:START
INFO:openbb_terminal.stocks.fundamental_analysis.sdk_helpers:{"INPUT": {"quarterly": "True", "ratios": "False", "source": "FinancialModelingPrep", "limit": "100", "symbol": "AAPL", "chart": "False"}, "VIRTUAL_PATH": "stocks.fa.balance", "CHART": false}
INFO:openbb_terminal.stocks.fundamental_analysis.fmp_model:START
INFO:openbb_terminal.stocks.fundamental_analysis.fmp_model:START
INFO:openbb_terminal.stocks.fundamental_analysis.fmp_model:END
INFO:openbb_terminal.stocks.fundamental_analysis.fmp_model:END
INFO:openbb_terminal.stocks.fundamental_analysis.sdk_helpers:END
INFO:openbb_terminal.stocks.fundamental_analysis.sdk_helpers:START
INFO:openbb_terminal.stocks.fundamental_analysis.sdk_helpers:{"INPUT": {"quarterly": "True", "ratios": "False", "source": "FinancialModelingPrep", "limit": "100", "symbol": "PLTR", "chart": "False"}, "VIRTUAL_PATH": "stocks.fa.balance", "CHART": false}
INFO:openbb_terminal.stocks.fundamental

In [13]:
df_final['ticker'].value_counts()

AAPL     100
AMZN     100
NVDA      98
NFLX      86
META      47
BABA      43
NIO       23
PLTR      14
FFIEW     13
RGTI      11
Name: ticker, dtype: int64

In [14]:
bulk_insert_data_from_dataframe(Balance, df_final)